### Лабораторная работа № 3. Вариант №4
##### Автор: Торопкин Р.А.
##### Проверил: к.т.н Балашева И.Ю. 

In [5]:
import numpy as np
import pandas as pd
from DefaultSymplex import DefaultSymplex

A_u = np.array([[1, 1, -1],
                [1, -5, 1]])
b_u = np.array([4, 5])
sign = ['<=', '>=']
c = np.array([2, -1, -5])

Приведение задачи к каноническому виду и формирование симплекс таблицы

In [6]:
def _ctc(A_u, b_u, c, sign):
    size = np.shape(A_u)
    new_A = np.concatenate((A_u, np.eye((size[0]))), axis=1)
    for i in range(0, len(sign)):
        if sign[i] == '>=':
            new_A[i] = np.dot(new_A[i], -1)
            new_A[i][len(new_A[i]) - size[0] + i] = 1
            b_u[i] = -b_u[i]
    c = np.concatenate((-c, np.zeros((1,size[0]))[0]))
    b_u = np.concatenate((b_u, [0])).reshape((3,1))
    index_arr = []
    columns_arr = [] 
    for i in range(0, size[0]):
        index_arr.append(f'x{len(new_A[0]) - size[0] + i + 1}')
    index_arr.append('Z')
    for i in range(0, len(new_A[0])):
        columns_arr.append(f'x{i+1}')
    columns_arr.append('b')
    table = np.vstack((new_A, c))
    table = np.hstack((table, b_u))
    df = pd.DataFrame(data=table, index=index_arr, columns=columns_arr)
    return df
    

In [7]:
df = _ctc(A_u, b_u, c, sign)
df

,x1,x2,x3,x4,x5,b
x4,1.0,1.0,-1.0,1.0,0.0,4.0
x5,-1.0,5.0,-1.0,0.0,1.0,-5.0
Z,-2.0,1.0,5.0,0.0,0.0,0.0


In [8]:
s = DefaultSymplex(table=df)
res = s.solve()

     x1   x2   x3   x4   x5    b
x4  1.0  1.0 -1.0  1.0  0.0  4.0
x5 -1.0  5.0 -1.0  0.0  1.0 -5.0
Z  -2.0  1.0  5.0  0.0  0.0  0.0
0 итерация 
     x1   x2   x3   x4   x5    b
x4  1.0  1.0 -1.0  1.0  0.0  4.0
x5 -1.0  5.0 -1.0  0.0  1.0 -5.0
Z  -2.0  1.0  5.0  0.0  0.0  0.0
Резрешающая строка: 1, Резрешающий столбец 1
Существует решение - True

1 итерация 
     x1   x2   x3   x4   x5    b
x1  1.0  1.0 -1.0  1.0  0.0  4.0
x5  0.0  6.0 -2.0  1.0  1.0 -1.0
Z   0.0  3.0  3.0  2.0  0.0  8.0
Резрешающая строка: 1, Резрешающий столбец 0
Существует решение - True



AttributeError: 'DefaultSymplex' object has no attribute '_xSolution'